---
cdt: 2024-09-05T12:30:33
title: Dataset EDA
desc: EDA on the dataset sample ID, time and wavelength in order to seperate the set into groups based on those dimensions so I can organise them into NetCDF datasets.
notes:
     - Doing SQL relations so that you can execute a query then write about AND build on that query. Relations dont work between scopes so everything has to be global.
project: "dataset_EDA"
execution_order: "000"
---


In [Migrating From DuckDB to NetCDF](export_data_from_db.ipynb), we found that in order to export the data to NetCDF from the SQL database its currently in, we need to ensure that the key dimensions align. What this means is that every sample needs to have the same wavelength and time values for rows/columns. This is because the magic of NetCDF/XArray is to share as many dimensions as possible. If I don't do this, the dataset will become too large for memory as it tries to fit all the different time points in. See 

Yet again we need to do some descriptive EDA on the dataset.

This can be performed within duckdb.

TODO:
- [x] what does sample_metadata look like?
- [x] what does chromatogram_spectra look like?
- [ ] distribution of:
  - [ ]  wavelengths
  - [ ] distribution of times
  - [x] timestep

# Summary

- outlier detection through observation of length of signal
  - To investigate the range of idx/mins, we observed the sample lengths, defined as the number of rows of 'mins_corrected' per sample 'id'.
  - We found 10 unique lengths ranging from 2087 to 15600, with 1 sample 2087 and one at 15600 - samples 29 and 87, respectively.
  - sample 29
    - sample 29 was determined to be an outlier because its runtime was cut too short and baseline was high relative to its neighbours
  - sample 87
    - sample 87 was determined to be an outlier with an observation frequency of 5Hz, double the sample frequency of 2.5Hz.
- dataset observation frequency
  - Total dataset observation frequncy bar outliers and minor errors was 2.5Hz
- outcomes
  - dataset is ready for resampling to common time coordinates.




In [ ]:
import duckdb as db
import polars as pl

db_path = "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/wines.db"

con = db.connect(db_path)


### Correcting Offset
